In [1]:
from elasticsearch import Elasticsearch
import json
import requests

import eslogin

import datetime
import time

import numpy as np
from numpy.random import randn
import pandas as pd
from scipy import stats
import matplotlib as mpl

import matplotlib.pyplot as plt
import seaborn as sns

import math

from sklearn.ensemble import RandomForestClassifier

import plotly.plotly as py
from plotly.graph_objs import *

%matplotlib inline
mpl.style.use('ggplot')

pd.options.display.max_rows = 10
pd.options.display.max_columns = 30

In [2]:
datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')

'2015-06-19 09:42:00'

In [3]:
es = Elasticsearch([{'host': eslogin.host, 'port': eslogin.port}],http_auth=(eslogin.user,eslogin.password))

In [4]:
es.count(index="events-*", body={'query': {'bool': {'must':[{'match': { 'et' : 'AD_CLICK' }}], 'must_not':[{'match': { 'fr' : 'true' }}]}}})['count']

11019

In [5]:
clicks = es.search(index="events-*", body={'query': {'bool': {'must':[{'match': { 'et' : 'AD_CLICK' }}], 'must_not':[{'match': { 'fr' : 'true' }}]}}}, size = 11019)

In [6]:
df_clicks = pd.DataFrame([datum['_source'] for datum in clicks['hits']['hits']])

In [7]:
df_clicks.shape

(11019, 46)

In [8]:
df_clicks.to_csv("../data/ad_clicks_150619.tab",sep='\t',encoding='utf-8')

In [9]:
clicks_uniq = df_clicks.loc[np.where(np.unique(df_clicks['created_at']))[0], :]

In [10]:
clicks_uniq.shape

(11018, 46)

In [11]:
es.count(index="events-*", body={'query': {'bool': {'must':[{'match': { 'et' : 'AD_SHOW' }}], 'must_not':[{'match': { 'fr' : 'true' }}]}}})['count']

5460523

In [12]:
randomInd = np.random.choice(range(5460523), size=20000, replace=False)

In [13]:
shows = []

In [20]:
for i in randomInd[14860:]:
    event = es.search(index="events-*", body={'query': {'bool': {'must':[{'match': { 'et' : 'AD_SHOW' }}], 'must_not':[{'match': { 'fr' : 'true' }}]}}}, from_ = i, size = 1)
    shows = shows + event['hits']['hits']

In [22]:
df_shows = pd.DataFrame([datum['_source'] for datum in shows])
df_shows.to_csv("../data/ad_shows_150619.tab",sep='\t',encoding='utf-8')

In [24]:
pd.options.display.max_rows = 3
pd.options.display.max_columns = 50
df_shows

,ai,ak,arch,br,ci,co,cr,created_at,cty,cvpc,dp,dt,edpt,ernc,et,ex,fr,id,ip,isp,li,lidp,pbep,pg,plfm,pp,pv,rgn,sdpt,ssp,ua,uid,vw_drtn,vw_perc
0,116,42,False,iPhone 0.0,NaN,USA,277,2015-05-14T00:58:05.931Z,Temple,NaN,0,iphone,0,0,AD_SHOW,NaN,False,zenit_034c6a51-b29e-4946-b14e-db3d8395f625_277,24.162.123.131,Time Warner Cable,197,0,70,0,ios,0,8.3,USA-TX,0,zenit_marketplace,Mozilla/5.0 (iPhone; CPU iPhone OS 8_3 like Ma...,NaN,0,0
1,54,127,False,Other 0.0,103,GBR,156,2015-05-08T12:27:27.389Z,Peterborough,NaN,0,other,5,0,AD_SHOW,NaN,False,zenit_d908f720-0936-4b83-85c2-4c85165e92c1_156,176.250.59.201,Sky Broadband,85,0,70,0,None,0,unknown,GBR-K3,13,zenit_marketplace,Manchester%20United/4.5.1 CFNetwork/711.1.16 D...,E4415AF4-A45A-4887-B7FF-C252860CC9D3,0,0
2,87,140,None,Safari 10.2.1.3442,0,NGA,0,2015-03-29T00:11:34.215Z,Lagos,NaN,10,other,0,7,AD_SHOW,NaN,False,rtb_4f449708-0eab-4f35-8db6-9571fcd61a53_22088,41.203.67.167,Globacom Ltd,0,0,70,0,None,0,unknown,NaN,0,NaN,Mozilla/5.0 (BB10; Kbd) AppleWebKit/537.35+ (K...,NaN,0,0
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [25]:
shows_uniq = df_shows.loc[np.where(np.unique(df_shows['created_at']))[0], :]
shows_uniq.shape

(19973, 34)